In [1]:
import requests
import datetime
import inspect
import json
from typing import (
    TypedDict, 
    List, Dict, Literal, 
    Callable, Optional, Any, 
    get_type_hints
)
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat.chat_completion_message import ChatCompletionMessage
from openai.types.chat.chat_completion_message_tool_call import ChatCompletionMessageToolCall


In [2]:
load_dotenv()

True

In [3]:
client = OpenAI()

In [4]:
class Memory:
    def __init__(self):
        self._messages: List[Dict[str, str]] = []
    
    def add_message(self, 
                    role: Literal['user', 'system', 'assistant', 'tool'], 
                    content: str,
                    tool_calls: dict=dict(),
                    tool_call_id=None)-> None:

        message = {
            "role": role,
            "content": content,
            "tool_calls": tool_calls,
        }

        if role == "tool":
            message = {
                "role": role,
                "content": content,
                "tool_call_id": tool_call_id,
            }

        self._messages.append(message)

    def get_messages(self) -> List[Dict[str, str]]:
        return self._messages

    def last_message(self) -> None:
        if self._messages:
            return self._messages[-1]

    def reset(self) -> None:
        self._messages = []

In [5]:
class Tool:
    def __init__(self, func:Callable):
        self.func = func
        self.name = func.__name__
        self.description = func.__doc__
        self.argument_types_map = get_type_hints(func)
        self.signature = inspect.signature(func)
        self.arguments = [
            {
                "name": key, 
                "type": self._infer_json_schema_type(value),
                "required": param.default == inspect.Parameter.empty
            } 
            for key, value in self.argument_types_map.items()
            if (param := self.signature.parameters.get(key))
        ]

    def dict(self):
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parallel_tool_calls": False,
                "parameters": {
                    "type": "object",
                    "properties": {
                        argument["name"]: {
                            "type": argument["type"],
                        }
                        for argument in self.arguments
                    },
                    "required": [
                        argument["name"] 
                        for argument in self.arguments 
                        if argument["required"]
                    ],
                    "additionalProperties": False,
                },
                "strict": True
            }
        }

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)
    
    def _infer_json_schema_type(self, arg_type: Any) -> str:
        if arg_type == bool:
            return "boolean"
        elif arg_type == int:
            return "integer"
        elif arg_type == float:
            return "number"
        elif arg_type == str:
            return "string"
        elif arg_type == list:
            return "array"
        elif arg_type == dict:
            return "object"
        elif arg_type is None:
            return "null"
        elif arg_type == datetime.date or arg_type == datetime.datetime:
            return "string"  # JSON Schema treats dates as strings
        else:
            return "string"  # Default to string if type is unknown

In [74]:
class StopReactLoopException(Exception):
    """
    Terminates ReAct loop
    """

In [119]:
TERMINATION_MESSAGE = "StopReactLoopException"

In [120]:
def termination()-> str:
    """Terminate the ReAct loop. If the agent thinks there's no further actions to take"""
    return TERMINATION_MESSAGE 

In [144]:
class Agent:
    """A tool-calling AI Agent"""

    def __init__(
        self,
        name:str = "Agent", 
        role:str = "Personal Assistant",
        instructions:str = "Help users with any question",
        model:str = "gpt-4o-mini",
        temperature:float = 0.0,
        tools:List[Tool] = [],
    ):
        self.name = name
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature
        self.client = OpenAI()
        self.tools = tools
        self.termination_message = TERMINATION_MESSAGE
        self._register_tool(Tool(termination))
        self.tool_map = {t.name:t for t in tools}
        self.openai_tools = [t.dict() for t in self.tools] if self.tools else None
        self.memory = Memory()

        self.memory.add_message(
            role="system",
            content=f"You're an AI Agent, your role is {self.role}, " 
                    f"and you need to {self.instructions} "
                    "You can answer multistep questions by sequentially calling functions. "
                    "You follow a pattern of of Thought and Action. "
                    "Create a plan of execution: "
                    "- Use Thought to describe your thoughts about the question you have been asked. "
                    "- Use Action to specify one of the tools available to you. "
                    "When you think it's over call the termination tool. "
                    "Never try to respond directly if the question needs a tool."
                    "The actions you have are the Tools: "
                    f"{self.openai_tools}",
        )
    
    def invoke(self, user_message: str, max_iter:int=3) -> str:
        self.memory.add_message(
            role="user",
            content=user_message,
        )
        try:
            self._react_loop(max_iter)
        except StopReactLoopException as e:
            print(f"Termninated loop with message: '{e!s}'")
            self._reason()

        return self.memory.last_message()

    def _react_loop(self, max_iter:int):
        for i in range(max_iter):
            self._reason()

            ai_message = self._get_completion(
                messages = self.memory.get_messages(),
                tools=self.openai_tools,
            )
            tool_calls = ai_message.tool_calls

            self.memory.add_message(
                role="assistant",
                content=ai_message.content,
                tool_calls=tool_calls,
            )

            if tool_calls:
                self._call_tools(tool_calls)

    def _reason(self):
        # No tools
        ai_message = self._get_completion(
            messages = self.memory.get_messages(),
        )
        tool_calls = ai_message.tool_calls

        self.memory.add_message(
            role="assistant",
            content=ai_message.content,
            tool_calls=tool_calls,
        )

    def _call_tools(self, tool_calls:List[ChatCompletionMessageToolCall]):
        for t in tool_calls:
            tool_call_id = t.id
            function_name = t.function.name
            args = json.loads(t.function.arguments)
            callable_tool = self.tool_map[function_name]
            result = callable_tool(**args)
            self.memory.add_message(
                role="tool", 
                content=str(result), 
                tool_call_id=tool_call_id
            )
            if result == TERMINATION_MESSAGE:
                raise StopReactLoopException

    def _get_completion(self, messages:List[Dict], tools:List=None)-> ChatCompletionMessage:
        response = self.client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=messages,
            tools=tools,
        )
        
        return response.choices[0].message

    def _register_tool(self, tool:Tool):
        self.tools.append(tool)



In [145]:
def power(base:float, exponent:float):
    """Exponentatiation: base to the power of exponent"""
    
    return base ** exponent

In [146]:
def sum(number_1:float, number_2:float):
    """Sum / Addition: Add two numbers"""
    
    return number_1 + number_2

In [147]:
funcs = [power, sum]
tools = [Tool(func) for func in funcs]

In [148]:
agent = Agent(
    tools=tools
)

In [149]:
agent.invoke("What's 2 to the power of 3? Then add 10 to the result")

Termninated loop with message: ''


{'role': 'assistant',
 'content': 'The final result of \\(2\\) to the power of \\(3\\) plus \\(10\\) is \\(18\\). If you have any more questions or need further assistance, feel free to ask!',
 'tool_calls': None}

In [150]:
agent.memory.get_messages()

[{'role': 'system',
  'content': 'You\'re an AI Agent, your role is Personal Assistant, and you need to Help users with any question You can answer multistep questions by sequentially calling functions. You follow a pattern of of Thought and Action. Create a plan of execution: - Use Thought to describe your thoughts about the question you have been asked. - Use Action to specify one of the tools available to you. When you think it\'s over call the termination tool. Never try to respond directly if the question needs a tool.The actions you have are the Tools: [{\'type\': \'function\', \'function\': {\'name\': \'power\', \'description\': \'Exponentatiation: base to the power of exponent\', \'parallel_tool_calls\': False, \'parameters\': {\'type\': \'object\', \'properties\': {\'base\': {\'type\': \'number\'}, \'exponent\': {\'type\': \'number\'}}, \'required\': [\'base\', \'exponent\'], \'additionalProperties\': False}, \'strict\': True}}, {\'type\': \'function\', \'function\': {\'name\'

In [22]:
def random_got_quote():
    response = requests.get("https://api.gameofthronesquotes.xyz/v1/random")
    return str(response.json())

In [ ]:
def get_audio_files(company_name:str):
    audio_files_map = {
        "apple": "apple.wav",
        "microsoft": "microsoft.wav",
        "google": "alphabet.wav",
        "amazon": "amazon.wav",
        "facebook": "meta.wav",
    }
    return audio_files_map[company_name]

In [ ]:
def transcribe_audio_file(audio_file:str):
    earnings_calls_2025 = {
        "apple.wav": "Apple reported strong Q1 2025 earnings, driven by record iPhone 16 sales and increased revenue from its subscription services. The company also announced advancements in AI-powered features for iOS and continued investment in augmented reality technologies.",
        "microsoft.wav": "Microsoft's revenue surged in 2025, fueled by Azure’s dominance in cloud computing and the expansion of AI-powered productivity tools in Office 365. The company also highlighted strong growth in gaming, with record engagement on Xbox and Game Pass.",
        "alphabet.wav": "Alphabet reported steady revenue growth, with Google Search and YouTube ad revenue rebounding after economic slowdowns. Google Cloud continues to gain enterprise customers, while the company’s AI models saw widespread adoption across industries.",
        "amazon.wav": "Amazon posted record-breaking earnings, with AWS leading cloud market growth. E-commerce sales remained strong, boosted by AI-driven logistics optimizations. The company also emphasized expansions in drone delivery and AI-powered Alexa services.",
        "meta.wav": "Meta’s earnings exceeded expectations, with continued revenue growth from Instagram and WhatsApp. The company doubled down on AI-powered content discovery and virtual reality adoption in the metaverse, with Quest headsets driving user engagement."
    }
    
